<img style="float: left;;" src='Figures/alinco.png' /></a>

# <center> <font color= #000047> Módulo 1: Crear una Red Neuronal Recurrente en TensorFlow 2.0



## Paso 1: Importar las librerías necesarias

In [1]:
import tensorflow as tf

from tensorflow.keras.datasets import imdb

In [2]:
tf.__version__

'2.2.0'

## Paso 2: Pre procesado de datos


### Configurar parámetros del dataset

In [3]:
number_of_words = 20000
max_len = 100

### Carga del dataset de IMDB

In [4]:
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=number_of_words)

C:\Users\uie70742\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\datasets\imdb.py:155: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
C:\Users\uie70742\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\datasets\imdb.py:156: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x_test, y_test = np.array(xs[idx:]), np.array(labels[idx:])


### Cortar secuencias de texto de la misma longitud

In [5]:
X_train = tf.keras.preprocessing.sequence.pad_sequences(X_train, maxlen=max_len)

In [6]:
X_test = tf.keras.preprocessing.sequence.pad_sequences(X_test, maxlen=max_len)

### Configurar parámetros de la capa de Embedding

In [7]:
vocab_size = number_of_words
vocab_size

20000

In [8]:
embed_size = 128

## Paso 3: Construir la Red Neuronal Recurrente

### Definir el modelo

In [9]:
model = tf.keras.Sequential()

### Añadir la capa de embedding

In [10]:
model.add(tf.keras.layers.Embedding(vocab_size, embed_size, input_shape=(X_train.shape[1],)))

### Añadir la capa de LSTM

- unidades: 128
- función de activación: tanh

In [11]:
#!pip install -U numpy==1.18.5 --user

In [12]:
model.add(tf.keras.layers.LSTM(units=128, activation='tanh'))

NotImplementedError: Cannot convert a symbolic Tensor (lstm/strided_slice:0) to a numpy array.

### Añadir la capa totalmente conectada de salida

- unidades: 1
- función de activación: sigmoid

In [ ]:
model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

### Compilar el modelo

In [ ]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

### Entrenar el modelo

In [ ]:
model.fit(X_train, y_train, epochs=3, batch_size=128)

### Evaluar el modelo

In [ ]:
test_loss, test_acurracy = model.evaluate(X_test, y_test)

In [ ]:
print("Test accuracy: {}".format(test_acurracy))

### Otro Ejemplo: Predecir el precio de las acciones de Google

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
dataset_train = pd.read_csv('Data/Google_Stock_Price_Train.csv')

training_set = dataset_train.iloc[:, 1:2].values
training_set.shape

In [ ]:
# Scaling should always be applied for 
# Deep Learning models

from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler()
training_set_scaled = sc.fit_transform(training_set)

In [ ]:
# the memory of RNN depends on the number of timesteps you select
# if timesteps = x then the output depends on the previous x inputs

# Create input set that consists of 60 dimensions
# hence, the output of current day will be based on 
# the prices of previous 60 days

X_train = []
y_train = []
for i in range(60, 1258):
    X_train.append(training_set_scaled[i - 60: i, 0])
    y_train.append(training_set_scaled[i, 0])
    
X_train, y_train = np.array(X_train), np.array(y_train)

# Reshape : convert our 2D array to 3D
(batch_size, timesteps) = X_train.shape
# This is done because the RNN class in Keras expects a 
# 3D Tensor
X_train = np.reshape(X_train, (batch_size, timesteps, 1))

In [ ]:
# Building RNN
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout

In [ ]:
# Initialise Sequential model
regressor = Sequential()

# units is the output dimensionality
# return sequences will return the sequence
# which will be required to the next LSTM 

# input shape will need only the last 2 dimensions
# of your input
################# 1st layer #######################
regressor.add(LSTM(units=50, return_sequences=True, 
                   input_shape=(timesteps, 1)))

# add Dropout to do regulariztion
# standard practise to use 20%
regressor.add(Dropout(0.2))

################# 2nd layer ######################
# After the first time, it's not required to 
# specify the input_shape

regressor.add(LSTM(units=50, return_sequences=True))
regressor.add(Dropout(0.2))


################# 3rd layer ######################
regressor.add(LSTM(units=50, return_sequences=True))
regressor.add(Dropout(0.2))
              

################# 4th layer ######################
# the last layer needn't return the sequence, so
# return_sequences will be False
              
regressor.add(LSTM(units=50))
regressor.add(Dropout(0.2))

################# 5th layer ######################
# Last layer would be the fully connected layer,
# or the Dense layer
#
# The last word will predict a single number
# hence units=1

regressor.add(Dense(units=1))

# Compiling the RNN
# The loss function for classification problem is 
# cross entropy, since this is a regression problem
# the loss function will be mean squared error
              
regressor.compile(optimizer='adam', loss='mean_squared_error')


In [ ]:
regressor.fit(X_train, y_train, epochs=100, batch_size=32)

In [ ]:
# Persisting the model for future use
from tensorflow.keras.models import load_model
# Save
regressor.save('regressor.hd5')
# Load
# model = load_model('regressor.hd5')

In [ ]:
# Preparing test data
dataset_test = pd.read_csv('Data/Google_Stock_Price_Test.csv')

real_stock_prices = dataset_test.iloc[:, 1:2].values

In [ ]:
# Preparing test dataset
#
# To predict the price for 3 Jan '17 (first financial
# day) we will need the price for 60 days before that, 
# so we will need both, the train and the test set to
# gather the required data

# 1. Concatenate training and test set

dataset_total = pd.concat((dataset_train['Open'], dataset_test['Open']),
                         axis=0) # axis=0 is concatenate rows

# 2. For each day from 1st Jan get 60 previous 
#    days' data

first_day_index = len(dataset_total) - len(dataset_test)
inputs = dataset_total[first_day_index - 60: ].values
inputs = inputs.reshape(-1, 1)

# 3. Scale the input

inputs = sc.transform(inputs)

In [ ]:
# Prepare the dataset to have 60 dimensions
X_test = []
for i in range(60, 80):
    X_test.append(inputs[i - 60: i, 0])
X_test = np.array(X_test)

# Reshape : convert our 2D array to 3D
(batch_size, timesteps) = X_test.shape
# This is done because the RNN class in Keras expects a 
# 3D Tensor
X_test = np.reshape(X_test, (batch_size, timesteps, 1))

In [ ]:
# Making prediction
predicted_stock_price = regressor.predict(X_test)
# inverse the scaling
predicted_stock_price = sc.inverse_transform(predicted_stock_price)

In [ ]:
# Visualing the results
plt.plot(real_stock_prices, color='red', 
         label='Real Google Stock Prices')
plt.plot(predicted_stock_price, color='green',
         label='Predicted Google Stock Prices')
plt.title('Google Stock Price Prediction using RNN')
plt.xlabel('Time')
plt.ylabel('Price')
plt.legend()
plt.show()

## Text Generation Using RNN

When working with text data tokens are words or characters and any network that can model the probability of the next token is called language model. A language model captures the statistical structure of the text. If we are training the neural network to predict the next character, it is called Character Level Model. Similarly, we can train the model to predict the next word, given a sequence of words called Word Level Models. We are implementing character level model.

<img src="Figures/63.png" alt="Grayscale Image" width="600">

### Implementing in Tensorflow
#### The Dataset
We will use a dataset which contains the works of Shakespeare.

In [ ]:
import tensorflow as tf
import numpy as np
#Download the dataset
path = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
#Explore the data
text = open(path, "r").read()
print(text[:200])

### Data Pre-Processing
Before training we need to map strings to numbers, extract partially overlapping sequences and pack them in a 3D numpy array of shape (sequences, maxlen, unique_characters). We one-hot encode the data.

In [ ]:
def process_text(file_path):
    text = open(file_path, 'rb').read().decode(encoding='utf-8')  # Read, then decode for py2 compat.
    vocab = sorted(set(text))  # The unique characters in the file
    # Creating a mapping from unique characters to indices and vice versa
    char2idx = {u: i for i, u in enumerate(vocab)}
    idx2char = np.array(vocab)
    text_as_int = np.array([char2idx[c] for c in text])
    return text_as_int, vocab, char2idx, idx2char


In [ ]:
def split_input_target(chunk):
    input_text, target_text = chunk[:-1], chunk[1:]
    return input_text, target_text

In [ ]:
def create_dataset(text_as_int, seq_length=100, batch_size=64, buffer_size=10000):
    char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
    dataset = char_dataset.batch(seq_length + 1, drop_remainder=True).map(split_input_target)
    dataset = dataset.shuffle(buffer_size).batch(batch_size, drop_remainder=True)
    return dataset


### Building the Network
The network is a single LSTM layer followed by a Dense classifier and softmax over all possible characters.

In [ ]:

def build_model(vocab_size, embedding_dim=256, rnn_units=1024, batch_size=64):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]),
        tf.keras.layers.LSTM(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.LSTM(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model


In [ ]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [ ]:
text_as_int, vocab, char2idx, idx2char = process_text(path)
dataset = create_dataset(text_as_int)
model = build_model(vocab_size=len(vocab))


### Compile and Train the model
Use the categorical_crossentropy loss to train the model as the targets are one-hot encoded.

In [ ]:
model.compile(optimizer='adam', loss=loss)
model.summary()


### Train the model

In [ ]:
history = model.fit(dataset, epochs=10)
model.save_weights("gen_text_weights.h5", save_format='h5')
# To keep this prediction step simple, use a batch size of 1
model = build_model(vocab_size=len(vocab), batch_size=1)
model.load_weights("gen_text_weights.h5")
model.summary()
 

## Generate text

In [ ]:
def generate_text(model, char2idx, idx2char, start_string, generate_char_num=1000, temperature=1.0):
    # Evaluation step (generating text using the learned model)
    # Low temperatures results in more predictable text, higher temperatures results in more surprising text.
    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)
    text_generated = []  # Empty string to store our results
    model.reset_states()
    for i in range(generate_char_num):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)    # remove the batch dimension
        predictions /= temperature
        # using a categorical distribution to predict the character returned by the model
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()
        # We pass the predicted character as the next input to the model along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], axis=0)
        text_generated.append(idx2char[predicted_id])
    return start_string + ''.join(text_generated)


In [ ]:
user_input = input("Write the beginning of the text, the program will complete it. Your input is: ")
generated_text = generate_text(model, char2idx, idx2char, start_string=user_input, generate_char_num=2000)
print(generated_text)